In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader, ExperimentConfig}
import com.flightdelay.data.loaders.FlightDataLoader

// Env Configuration
val args: Array[String] = Array("jupyter")
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)
implicit val experimentConfig: ExperimentConfig = configuration.experiments(0)

val spark = SparkSession.builder()
  .config(sc.getConf)
  .config("spark.eventLog.enabled", "true")
  .config("spark.eventLog.dir", s"${configuration.common.output.basePath}/spark-events")  // ex: "file:/tmp/spark-events" ou "hdfs:///spark-events"
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark


args = Array(jupyter)
configuration = AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...


AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...

In [3]:
val joinedDataPath = s"${configuration.common.output.basePath}/Experience-local-3-3/data/join_exploded_train_prepared.parquet"
val joinedData = spark.read.parquet(joinedDataPath)

println("Joined DF Count: ", joinedData.count())


(Joined DF Count: ,117728)


joinedDataPath = /home/jovyan/work/output/Experience-local-3-3/data/join_exploded_train_prepared.parquet
joinedData = [CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 164 more fields]


[CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 164 more fields]

In [4]:
joinedData.printSchema

root
 |-- CRS_ARR_TIME: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- D1: integer (nullable = true)
 |-- D2_15: integer (nullable = true)
 |-- D2_30: integer (nullable = true)
 |-- D2_45: integer (nullable = true)
 |-- D2_60: integer (nullable = true)
 |-- D2_90: integer (nullable = true)
 |-- D3: integer (nullable = true)
 |-- D4: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- UTC_CRS_DEP_TIME: string (nullable = true)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- depHour: integer (nullable = true)
 |-- feature_arrival_time_period: string (nullable = true)
 |-- feature_departure_hour_rounded: long (nullable 

In [5]:
import com.flightdelay.features.pipelines.ConfigurationBasedFeatureExtractorPipeline
import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}


val handleInvalid = "keep"

val configPipeline = ConfigurationBasedFeatureExtractorPipeline(
  featureConfig = experimentConfig.featureExtraction,
  target = "is_delayed",
  handleInvalid = handleInvalid
)

val preprocessed = configPipeline.preprocessBooleans(joinedData)


handleInvalid = keep
configPipeline = com.flightdelay.features.pipelines.ConfigurationBasedFeatureExtractorPipeline@2682233f
preprocessed = [CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 164 more fields]


[CRS_ARR_TIME: string, CRS_DEP_TIME: int ... 164 more fields]

In [7]:
val stages = configPipeline.buildPipelineStages(preprocessed, experimentConfig)


[INFO] [ConfigurationBasedFeatureExtractorPipeline] Feature Distribution:
[INFO]   - StringIndexer features: 37
[INFO]     OP_CARRIER_AIRLINE_ID, destination_weather_feature_pressure_bucket_h1, destination_weather_feature_pressure_bucket_h2, destination_weather_feature_pressure_bucket_h3, origin_weather_feature_pressure_bucket_h1, ... (32 more)
[DEBUG] [ConfigurationBasedFeatureExtractorPipeline] Categorical Features Cardinality:
[DEBUG]   [ 0] OP_CARRIER_AIRLINE_ID                              :     16 distinct values
[DEBUG]   [ 1] destination_weather_feature_pressure_bucket_h1     :      5 distinct values
[DEBUG]   [ 2] destination_weather_feature_pressure_bucket_h2     :      5 distinct values
[DEBUG]   [ 3] destination_weather_feature_pressure_bucket_h3     :      5 distinct values
[DEBUG]   [ 4] origin_weather_feature_pressure_bucket_h1          :      5 distinct values
[DEBUG]   [ 5] origin_weather_feature_pressure_bucket_h2          :      5 distinct values
[DEBUG]   [ 6] origi

stages = Array(strIdx_e6b9a46317e8, strIdx_d105aab650dc, imputer_c1e6ff586e63, VectorAssembler: uid=vecAssembler_ee6dbcfc16a3, handleInvalid=keep, numInputCols=93, vecIdx_a9a4fc4c58e1)


Array(strIdx_e6b9a46317e8, strIdx_d105aab650dc, imputer_c1e6ff586e63, VectorAssembler: uid=vecAssembler_ee6dbcfc16a3, handleInvalid=keep, numInputCols=93, vecIdx_a9a4fc4c58e1)

In [8]:
val pipeline = new Pipeline().setStages(stages)
val model = pipeline.fit(preprocessed)
val transformed = configPipeline.postProcess(model.transform(preprocessed))

pipeline = pipeline_955d851782db
model = pipeline_955d851782db
transformed = [features: vector, label: double]


[features: vector, label: double]

In [9]:
transformed.show(1,10000, true)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 features | (93,[0,7,8,15,29,30,32,33,34,35,36,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,74,75,76,77,78,79,80,81,82,83,84,85,86],[8.0,9.0,13.0,2.0,3.0,1.0,3.0,3.0,3.0,1.0,6.0,7.0,10.0,8.0,5.0,7.0,7.0,82.0,76.0,74.0,18.0,17.0,16.0,304.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,1326.0,9.97918413653628,10.14760566158702,10.408580234639093,10.74273406941625,11.12462477486492,11.047547093784628]) 
 label    | 1.0                                                                 

In [9]:
print(transformed.count())

117728